# Packages 

In [7]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
import numpy as np
import polars as pl
from src.config import raw_data_dir, raw_data_session_id_dir, task1_train_eval_dir

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config 

# Download data

In [20]:
!mkdir ../{raw_data_dir}
!mkdir ../{raw_data_session_id_dir}
!mkdir ../{task1_train_eval_dir}

mkdir: cannot create directory ‘../data/raw_data’: File exists
mkdir: cannot create directory ‘../data/raw_data_session_id’: File exists
mkdir: cannot create directory ‘../data/task1_train_eval’: File exists


In [8]:
!aicrowd login

Please login here: https://api.aicrowd.com/auth/FBlx5GdYycMF1E0I_CN1HsGicHjSzoMpIs1rC2Xjj8M
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 870: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/FBlx5GdYycMF1E0I_CN1HsGicHjSzoMpIs1rC2Xjj8M'
API Key valid
Gitlab access token valid
Saved details successfully!


In [13]:
# !aicrowd dataset download --challenge task-1-next-product-recommendation

In [15]:
!aicrowd dataset list --challenge task-1-next-product-recommendation 
# ../{raw_data_dir}

                          Datasets for challenge #1116                          
┌───┬──────────────────────────────┬───────────────────────────────┬───────────┐
│ # │ Title                        │ Description                   │      Size │
├───┼──────────────────────────────┼───────────────────────────────┼───────────┤
│ 0 │ Amazon KDD 2023 Test Data -  │ Phase 1 test sessions for KDD │  19.37 MB │
│   │ Phase                        │ Cup 2023                      │           │
│   │ 1sessions_test_task1.csv     │                               │           │
│ 1 │ Amazon KDD 2023 Test Data -  │ Phase 1 test sessions for KDD │   1.92 MB │
│   │ Phase                        │ Cup 2023                      │           │
│   │ 1sessions_test_task2.csv     │                               │           │
│ 2 │ Amazon KDD 2023 Test Data -  │ Phase 1 test sessions for KDD │   2.67 MB │
│   │ Phase                        │ Cup 2023                      │           │
│   │ 1sessions_test_task3.c

# Preprocess data 

# Task1 

In [2]:
train_pl = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_train.csv'))
task1_pl = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_test_task1.csv'))
task2_pl = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_test_task2.csv'))
task3_pl = pl.scan_csv(os.path.join('..', raw_data_dir, 'sessions_test_task3.csv'))
product_pl = pl.scan_csv(os.path.join('..', raw_data_dir, 'products_train.csv'))

In [3]:
length_lst = []
for df in [train_pl, task1_pl, task2_pl, task3_pl, product_pl]:
    print()
    print()
    print(df.schema)
    shape = df.collect().shape
    length_lst.append(shape[0])
    print(shape)
    print()
    print(df.head(3).collect())
    print()
    print()



{'prev_items': Utf8, 'next_item': Utf8, 'locale': Utf8}
(3606249, 3)

shape: (3, 3)
┌───────────────────────────────────┬────────────┬────────┐
│ prev_items                        ┆ next_item  ┆ locale │
│ ---                               ┆ ---        ┆ ---    │
│ str                               ┆ str        ┆ str    │
╞═══════════════════════════════════╪════════════╪════════╡
│ ['B09W9FND7K' 'B09JSPLN1M']       ┆ B09M7GY217 ┆ DE     │
│ ['B076THCGSG' 'B007MO8IME' 'B08M… ┆ B001B4THSA ┆ DE     │
│ ['B0B1LGXWDS' 'B00AZYORS2' 'B0B1… ┆ B0767DTG2Q ┆ DE     │
└───────────────────────────────────┴────────────┴────────┘




{'prev_items': Utf8, 'locale': Utf8}
(316971, 2)

shape: (3, 2)
┌───────────────────────────────────┬────────┐
│ prev_items                        ┆ locale │
│ ---                               ┆ ---    │
│ str                               ┆ str    │
╞═══════════════════════════════════╪════════╡
│ ['B08V12CT4C' 'B08V1KXBQD' 'B01B… ┆ DE     │
│ ['B00R9R5ND6' 'B00R9RZ

In [4]:
length_lst

[3606249, 316971, 34691, 56423, 1551057]

In [15]:
train_pl = train_pl.with_columns(
    pl.Series(name='session_id', values=range(length_lst[0]))
    , pl.Series(name='random', values=np.random.uniform(size=length_lst[0]))
)
task1_pl= task1_pl.with_columns(
    pl.Series(name='session_id', values=range(length_lst[0],sum(length_lst[:2])))
)
task2_pl= task2_pl.with_columns(
    pl.Series(name='session_id', values=range(sum(length_lst[:2]),sum(length_lst[:3])))
)
task3_pl= task3_pl.with_columns(
    pl.Series(name='session_id', values=range(sum(length_lst[:3]),sum(length_lst[:4])))
)

In [16]:
# assert train_pl.select('session_id').max().collect()<200

In [17]:
train_pl.select('session_id').max().collect()

session_id
i64
3606248


In [53]:
task1_pl.select('session_id').min().collect()

session_id
i64
3606249


In [54]:
task2_pl.select('session_id').min().collect()

session_id
i64
3923220


In [55]:
task3_pl.select('session_id').min().collect()

session_id
i64
3957911


## Save data with session_id 

In [18]:
eval_df = train_pl.filter(pl.col('random')<=0.1).select(pl.all().exclude('random'))
train_df = train_pl.filter(pl.col('random')>0.1).select(pl.all().exclude('random'))

In [19]:
eval_df.collect().shape

(359776, 4)

In [20]:
train_df.collect().shape

(3246473, 4)

In [21]:
train_pl.collect().shape

(3606249, 5)

In [22]:
train_df.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'))
eval_df.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'))

In [66]:

task1_pl.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'))
task2_pl.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task2.parquet'))
task3_pl.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'))

## Production data 

In [82]:
value_counts = product_pl.with_columns(pl.count().over('id').alias('cnt')).select(['id', 'cnt']).unique().collect().to_pandas()['cnt'].value_counts()

In [84]:
value_counts

cnt
1    1311106
2      72977
3      16368
4       6738
5       2975
6        511
Name: count, dtype: int64

In [86]:
value_counts/value_counts.sum()

cnt
1    0.929417
2    0.051732
3    0.011603
4    0.004776
5    0.002109
6    0.000362
Name: count, dtype: float64

**92% of products only in a single country; the rests are in more than one country**

### A example 

In [72]:
product_pl.with_columns(pl.count().over('id').alias('cnt')).sort(['cnt'], descending=True).head().collect()

id,locale,title,price,brand,color,size,model,material,author,desc,cnt
str,str,str,f64,str,str,str,str,str,str,str,u32
"""B0B9W7W8JD""","""FR""","""RAVIAD Câble U…",6.99,"""RAVIAD""","""Argent""","""0.3M""","""H011-SI-030""",null,null,"""【Compatibilité…",6
"""B09YMV3TXX""","""DE""","""Spigen Ultra H…",16.99,"""Spigen""","""Crystal Clear""",null,"""ACS04726""","""Polycarbonat""",null,"""Die Konstrukti…",6
"""B005EJH6RW""","""DE""","""Amazon Basics …",9.69,"""Amazon Basics""","""Schwarz""","""1-Pack""","""MSU0939""",null,null,"""High-definitio…",6
"""B07ZF8WFZN""","""IT""","""Cavo da HDMI a…",11.99,"""BENFEI""","""1 Confezioni""","""0,9 metri""","""000282black""",null,null,"""18 MESI DI GAR…",6
"""B075GQ87PV""","""JP""","""レゴ(LEGO) クラシック…",2345.0,"""レゴ(LEGO)""",null,null,"""10713""",null,null,"""かしこく収納""",6


In [74]:
product_pl.filter(pl.col('id')=='B0B9W7W8JD').collect()

id,locale,title,price,brand,color,size,model,material,author,desc
str,str,str,f64,str,str,str,str,str,str,str
"""B0B9W7W8JD""","""DE""","""USB C Kabel Ku…",6.99,"""RAVIAD""","""Silver""","""0.3M""","""H011-SI-030""",null,null,"""【Beidseitig nu…"
"""B0B9W7W8JD""","""JP""","""RAVIAD USB Typ…",699.0,"""RAVIAD""","""シルバー""","""0.3M""","""H011-SI-030""",null,null,"""【卓越した耐久性】高耐久を持…"
"""B0B9W7W8JD""","""UK""","""Short USB C Ca…",5.99,"""RAVIAD""","""Silver""","""0.3M""","""H011-SI-030""",null,null,"""【24-Month Warr…"
"""B0B9W7W8JD""","""ES""","""RAVIAD Cable U…",6.99,"""RAVIAD""","""Silver""","""0.3M""","""H011-SI-030""",null,null,"""【Carga Rápida …"
"""B0B9W7W8JD""","""FR""","""RAVIAD Câble U…",6.99,"""RAVIAD""","""Argent""","""0.3M""","""H011-SI-030""",null,null,"""【Compatibilité…"
"""B0B9W7W8JD""","""IT""","""RAVIAD Cavo US…",6.99,"""RAVIAD""","""Argento""","""0.3M""","""H011-SI-030""",null,null,"""【Connettore Re…"


### Save product info 

In [87]:
product_pl = product_pl.with_columns(
    pl.Series(name='unique_id', values=range(length_lst[-1]))
)

In [88]:
product_pl.schema

{'id': Utf8,
 'locale': Utf8,
 'title': Utf8,
 'price': Float64,
 'brand': Utf8,
 'color': Utf8,
 'size': Utf8,
 'model': Utf8,
 'material': Utf8,
 'author': Utf8,
 'desc': Utf8,
 'unique_id': Int64}

In [89]:
product_pl.collect().write_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'products_train.parquet'))


# Task2 